# Nivel del Mar

Para ver el concepto de Nivel del Mar y Cambio Climático revise la [Introduccion](Introduction.ipynb).
+ Mapas de Nivel del Mar de años recientes en Magdalena.
+ Gráfico de Tendencia de Nivel del Mar en Santa Marta y Cartagena. 

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Mapas del Nive del Mar

In [44]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
    
def nivel_mar(year):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    img = mpimg.imread(r'./Datos/Nivel del Mar/Nivel_Mar_'+str(year)+'.png')
    imgplot = ax.imshow(img)
    ax.axis('off')
    #fig.suptitle('test title', fontsize=20)
    plt.title('Nivel del Mar en Noviembre de '+str(year), fontsize=15)
    plt.show()

In [45]:
import ipywidgets as widgets
import time

a = widgets.IntSlider(min=2014,max=2018,step=1, description='Año:')
ui = widgets.HBox([a])
def f(a):
    nivel_mar(a)
    
out = widgets.interactive_output(f, {'a': a})

display(ui, out)

Output()

## Gráfico de Tendencia del Nivel del Mar 

In [5]:
import plotly.express as px
import pandas as pd
import numpy as np
import datetime as dt

def get_trend(ciudad):
    df = pd.read_csv(r".\Datos\Nivel del Mar\\"+str(ciudad)+".csv")
    df.Tiempo = pd.to_datetime(df['Tiempo'], format = '%d/%m/%Y')
    
    inicio = df['Tiempo'].dt.year.iloc[0]
    final = df['Tiempo'].dt.year.iloc[-1]

    fig = px.line(df, x="Tiempo", y=["Nivel del Mar"], template="seaborn")
    fig.update_traces(mode="markers+lines", hovertemplate=None)
    fig.update_layout(hovermode="x unified", yaxis_title="Nivel Medio del Mar [mm]", xaxis_title="Tiempo",
                     title_text='Nivel medio del Mar en '+str(ciudad)+' '+str(inicio)+'-'+str(final), title_x=0.22)
  
    fig.show()

In [6]:
import ipywidgets as widgets

a = widgets.Dropdown(options=['Santa_Marta', 'Cartagena'], description='Tipo:')
ui = widgets.HBox([a])
def f(a):
    get_trend(a)

out = widgets.interactive_output(f, {'a': a})

display(ui, out)

Output()